In [1]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
import yaml
import os
import matplotlib.pyplot as plt

from gammapy.maps import MapAxis, WcsGeom, Map
from gammapy.data import DataStore
from gammapy.makers import MapDatasetMaker, SafeMaskMaker, FoVBackgroundMaker
from gammapy.modeling.models import (
    FoVBackgroundModel,
    Models,
)
from gammapy.modeling import Fit
from gammapy.datasets import MapDataset
from gammapy.irf import Background3D

import warnings
warnings.filterwarnings('ignore')

In [4]:
# loading general parameters
hessera = 'hess1u'
source_catalog = "Sgr A*"

with open("/home/vault/caph/mppi062h/repositories/syserror_3d_bkgmodel/general_config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
conf=cfg['conf']
zen_bins=cfg['zen_bins']

In [5]:
basedir = f'$FITS_PROD/{hessera}/std_{conf}_fullEnclosure'
ds = DataStore.from_dir(basedir, f'hdu-index-bg-latest-fov-radec.fits.gz', f'obs-index-bg-latest-fov-radec.fits.gz')
obs_table = ds.obs_table

# run selection
mask_source = obs_table['OBJECT'] == source_catalog
mask_tel = obs_table['N_TELS'] == cfg['mask_ntels']
mask_zenpnt = obs_table['ZEN_PNT'] < cfg['mask_zenpnt']
mask_eventcount = obs_table['EVENT_COUNT'] > cfg['mask_eventcount']
mask_ontime = obs_table['ONTIME'] > cfg['mask_ontime']
mask_final = mask_source & mask_tel & mask_zenpnt & mask_eventcount & mask_ontime 
runlist = np.asarray(obs_table['OBS_ID'][mask_final])

In [6]:
np.savetxt(f'runlist_{hessera}.txt', runlist)